In [3]:
import tkinter as tk
from tkinter import filedialog

def open_file_dialog():
    # Initialize Tkinter root
    root = tk.Tk()
    root.withdraw()  # Hide the root window

    # Open file dialog and allow selection of .mp files
    file_path = filedialog.askopenfilename(
        title="Select a .mp file",
        filetypes=[("MP files", "*.mp"), ("All files", "*.*")]
    )
    
    # Return the selected file path
    return file_path

# Run the file dialog and get the selected file path
selected_file_path = open_file_dialog()

# Check the selected file path
print(f"Selected file: {selected_file_path}")


Selected file: /Users/jamesmclean/Documents/Uni/Laidlaw/Ralf_Research/my_data/2024_08_01/001_JM_1A_I.mp


In [4]:
import h5py
import numpy as np

# Ensure a file has been selected
if selected_file_path:
    try:
        # Open the file using h5py and extract frames
        with h5py.File(selected_file_path, "r") as hdf_file:
            # Check available keys to verify the file structure
            print("Keys in the file:", list(hdf_file.keys()))
            
            # Load the frames dataset
            frames = hdf_file["movie/frame"][:]
        
        # Print the shape of the frames to verify loading
        print(f"Frames loaded successfully with shape: {frames.shape}")
    except Exception as e:
        print(f"Error loading frames: {e}")
else:
    print("No file selected.")


Keys in the file: ['analysis', 'display', 'format_version_number', 'movie']
Frames loaded successfully with shape: (11988, 34, 128)


In [5]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed

def display_frame(frame_index, frames):
    """
    Display a single frame.

    Parameters:
        frame_index (int): Index of the frame to display.
        frames (np.ndarray): Array of frames.
    """
    plt.figure(figsize=(8, 4))
    plt.imshow(frames[frame_index], cmap='gray')
    plt.title(f"Frame {frame_index}")
    plt.axis('off')
    plt.show()

def create_frame_slider(frames):
    """
    Create an interactive slider to view frames.

    Parameters:
        frames (np.ndarray): Array of frames.
    """
    interact(display_frame, frame_index=widgets.IntSlider(min=0, max=len(frames)-1, step=1, value=0), frames=fixed(frames))

# Create an interactive slider if frames are loaded
if 'frames' in locals():
    create_frame_slider(frames)
else:
    print("Frames not loaded.")


interactive(children=(IntSlider(value=0, description='frame_index', max=11987), Output()), _dom_classes=('widg…

In [6]:
import h5py
import numpy as np

# Ensure a file has been selected
if selected_file_path:
    try:
        # Open the file using h5py and extract frames
        with h5py.File(selected_file_path, "r") as hdf_file:
            # Check available keys to verify the file structure
            print("Keys in the file:", list(hdf_file.keys()))
            
            # Load the frames dataset
            frames = hdf_file["movie/frame"][:]
        
        # Print the shape of the frames to verify loading
        print(f"Frames loaded successfully with shape: {frames.shape}")
    except Exception as e:
        print(f"Error loading frames: {e}")
else:
    print("No file selected.")

# Implementing ratiometric image processing
def process_ratiometric_images(frames):
    """
    Process frames to create ratiometric images.

    Parameters:
        frames (np.ndarray): Array of raw frames.

    Returns:
        np.ndarray: Array of ratiometric images.
    """
    # Assuming background subtraction is needed; we take the mean of initial frames as background
    background = np.mean(frames[:10], axis=0)

    # Subtract background from all frames
    subtracted_frames = frames - background

    # Calculate ratiometric images
    # Example: Ratio between current frame and mean of initial frames for normalization
    ratiometric_images = subtracted_frames / (background + 1e-5)  # Adding a small value to prevent division by zero

    # Normalize the ratiometric images to the range of 0 to 255 for visualization
    ratiometric_images = np.clip(ratiometric_images, 0, None)  # Ensure no negative values
    ratiometric_images = (ratiometric_images / np.max(ratiometric_images) * 255).astype(np.uint8)
    
    return ratiometric_images

# Process the frames to obtain ratiometric images
if 'frames' in locals():
    ratiometric_images = process_ratiometric_images(frames)
    print(f"Ratiometric images processed with shape: {ratiometric_images.shape}")


Keys in the file: ['analysis', 'display', 'format_version_number', 'movie']
Frames loaded successfully with shape: (11988, 34, 128)
Ratiometric images processed with shape: (11988, 34, 128)


In [7]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, fixed

def display_images(frame_index, frames, ratiometric_frames, mode='native'):
    """
    Display a single frame.

    Parameters:
        frame_index (int): Index of the frame to display.
        frames (np.ndarray): Array of frames.
        ratiometric_frames (np.ndarray): Array of ratiometric frames.
        mode (str): Display mode ('native' or 'ratiometric').
    """
    plt.figure(figsize=(8, 4))
    if mode == 'native':
        plt.imshow(frames[frame_index], cmap='gray')
        plt.title(f"Native Image - Frame {frame_index}")
    else:
        plt.imshow(ratiometric_frames[frame_index], cmap='gray')
        plt.title(f"Ratiometric Image - Frame {frame_index}")
    plt.axis('off')
    plt.show()

def create_interactive_viewer(frames, ratiometric_frames):
    """
    Create an interactive viewer to toggle between native and ratiometric images.

    Parameters:
        frames (np.ndarray): Array of native frames.
        ratiometric_frames (np.ndarray): Array of ratiometric frames.
    """
    interact(display_images, 
             frame_index=widgets.IntSlider(min=0, max=len(frames)-1, step=1, value=0), 
             frames=fixed(frames),
             ratiometric_frames=fixed(ratiometric_frames),
             mode=widgets.RadioButtons(options=['native', 'ratiometric'], value='native', description='Mode'))

# Create the interactive viewer if frames are loaded
if 'frames' in locals() and 'ratiometric_images' in locals():
    create_interactive_viewer(frames, ratiometric_images)
else:
    print("Frames or ratiometric images not loaded.")


interactive(children=(IntSlider(value=0, description='frame_index', max=11987), RadioButtons(description='Mode…